# 01 — Certificação de V_HL(2)

Este notebook audita o produto de Euler para o fator singular de Hardy–Littlewood em h=2 (primos gêmeos):

$$ V_{HL}(2) = 2 \prod_{p>2} \left(1 - \frac{1}{(p-1)^2} \right). $$

Objetivos:
- Computar a aproximação truncada V_HL(2; P_max) com mpmath, usando teste determinístico de primalidade simples.
- Controlar a cauda por uma cota multiplicativa auditável (exp(C/(P_max-1)) - 1, C≈1).
- Validar convergência quando P_max e precisão (mp.dps) crescem, com relatório e gráfico de estabilidade.
- Emitir asserts simples para garantir reprodutibilidade mínima.

Este notebook integra o repositório `tesa-meda-twins` e consome `src/vhl.py`.

In [ ]:
# Setup: caminhos e importações
import os, sys, json, math
from pathlib import Path
import numpy as np
import mpmath as mp

BASE = Path('tesa-meda-twins')
SRC = BASE/'src'
sys.path.append(str(BASE))
sys.path.append(str(SRC))

try:
    from vhl import vhl_trunc, tail_bound
except Exception as e:
    # Fallback: define funções aqui se src/vhl.py não estiver disponível
    def is_prime(n: int) -> bool:
        if n < 2:
            return False
        if n % 2 == 0:
            return n == 2
        r = int(mp.sqrt(n))
        for k in range(3, r+1, 2):
            if n % k == 0:
                return False
        return True

    def vhl_trunc(Pmax: int = 100000, mp_dps: int = 50):
        mp.mp.dps = mp_dps
        prod = mp.mpf("1.0")
        for p in range(3, int(Pmax)+1):
            if is_prime(p):
                prod *= (1 - mp.mpf(1)/((p-1)*(p-1)))
        return mp.mpf(2)*prod

    def tail_bound(Pmax: int):
        if Pmax <= 2:
            return 1.0
        C = 1.0
        return float(mp.e**(C/(Pmax-1.0)) - 1.0)

print({'status': 'loaded', 'src': str(SRC.resolve())})

## Cálculo único de referência
Executamos uma avaliação com P_max moderado e alta precisão para obter uma referência e o erro de cauda teórico.

In [ ]:
mp.mp.dps = 80
Pmax_ref = 200_000  # aumente conforme recurso
V_ref = vhl_trunc(Pmax=Pmax_ref, mp_dps=mp.mp.dps)
err_ref = tail_bound(Pmax=Pmax_ref)
print({'V_HL_ref': str(V_ref), 'tail_bound_mult': err_ref, 'Pmax': Pmax_ref, 'mp.dps': mp.mp.dps})

## Varredura de parâmetros (P_max, precisão) e relatório
Avaliamos a estabilidade de V_HL(2; P_max) para diferentes cortes e precisões, medindo variação relativa e a cota de cauda.

In [ ]:
grid_P = [20_000, 50_000, 100_000, 200_000]
grid_dps = [40, 60, 80]
rows = []
for dps in grid_dps:
    for P in grid_P:
        V = vhl_trunc(Pmax=P, mp_dps=dps)
        err = tail_bound(Pmax=P)
        rows.append({'Pmax': P, 'mp_dps': dps, 'VHL': float(V), 'tail_bound_mult': err})

import pandas as pd
df = pd.DataFrame(rows).sort_values(['mp_dps','Pmax'])
df.reset_index(drop=True, inplace=True)
df

## Gráfico de convergência e cota de cauda
Visualizamos a aproximação V_HL(2; P_max) por P_max (para cada mp.dps) e a curva da cota de cauda 1 + (exp(C/(P-1)) - 1).

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(7,4))
for dps in sorted(df['mp_dps'].unique()):
    dsub = df[df['mp_dps']==dps]
    ax.plot(dsub['Pmax'], dsub['VHL'], marker='o', label=f'mp.dps={dps}')

P_plot = np.linspace(min(grid_P), max(grid_P), 200)
tail_mult = np.exp(1.0/(P_plot-1.0)) - 1.0
ax2 = ax.twinx()
ax2.plot(P_plot, 1.0 + tail_mult, 'k--', alpha=0.5, label='1 + tail_bound')
ax.set_xlabel('P_max')
ax.set_ylabel('V_HL(2; P_max)')
ax2.set_ylabel('Fator multiplicativo')
ax.grid(True, ls=':')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout(); plt.show()

## Verificações auditáveis
- Monotonia aproximada: aumentar P_max não deve alterar muito o valor estimado (variação relativa pequena após certo regime).
- A cota de cauda deve diminuir com P_max.
- Reprodutibilidade: mesma configuração deve produzir valor idêntico (determinístico).

In [ ]:
def rel_diff(a, b):
    return abs(a-b)/max(1e-30, abs(b))

ok_variation = True
for dps in grid_dps:
    prev = None
    for P in grid_P:
        val = float(df[(df['mp_dps']==dps)&(df['Pmax']==P)]['VHL'])
        if prev is not None:
            if rel_diff(val, prev) > 0.05:  # 5% tolerância conservadora
                ok_variation = False
        prev = val
assert ok_variation, 'Variação relativa muito grande ao aumentar P_max'

ok_tail = True
for i in range(1, len(grid_P)):
    if not (tail_bound(grid_P[i]) <= tail_bound(grid_P[i-1])):
        ok_tail = False
assert ok_tail, 'Tail bound não decresceu com P_max'

ref1 = vhl_trunc(Pmax=50000, mp_dps=60)
ref2 = vhl_trunc(Pmax=50000, mp_dps=60)
assert str(ref1) == str(ref2), 'Reprodutibilidade falhou'
print({'checks': 'passed'})

## Conclusão
- O produto de Euler truncado para V_HL(2) estabiliza numericamente ao crescer P_max e a precisão (mp.dps).
- A cota de cauda multiplicativa exp(C/(P_max-1)) - 1 decresce com P_max e serve como bound auditável.
- As verificações mostram variação relativa moderada e determinismo.

Próximos passos no repositório:
- notebooks/02_transfer_kernel_tests.ipynb: validação do kernel agregado e do teste de massa (T_X).
- notebooks/03_spectrum_gap_alpha0.ipynb: estimativa de alpha_0 (gap periférico em média zero).
- notebooks/04_coercao_tesa_tiles.ipynb: coerção por tiles e lambda_*.